# Large data handling example

This is an example of how to read and render a 'large' quantity of data, whether locally or remote, without having to copy and/or rechunk the data into a more suitable data format. Data chosen is from a Neuropixels probe's local field potential (LFP) data from one of the Allen Institute for Brain Science's AllenSDK example sessions.

## Obtain LFP data

First download the required data using the AllenSDK. This will be cached locally, so the first time this notebook is run will be slow as it downloads the data, and the second and subsequent times will be much faster. The data file is an NWB file which is an HDF5 file with a particular internal format. The data file is downloaded locally for ease of experimenting with, but this workflow also supports the data file begin accessed remotely so that it could, for example, be in an s3 bucket and we would only read the chunks that we need.

In [ ]:
import warnings; warnings.simplefilter('ignore')
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import os

Store the AllenSDK cache somewhere sensible.

In [ ]:
local_cache_dir = os.path.expanduser(os.path.join("~", "allensdk_cache"))

In [ ]:
cache = EcephysProjectCache.from_warehouse(manifest=os.path.join(local_cache_dir, "manifest.json"))
sessions = cache.get_session_table()
sessions.head()

In [ ]:
# We take the very first session.
session_id = sessions.index.values[0]
session = cache.get_session_data(session_id)
session

Each session contains a number of probes.

In [ ]:
session.probes

We will take the first probe.

In [ ]:
probe_id = session.probes.index.values[0]
# lfp = session.get_lfp(probe_id) # This will load 2 GB of LFP data

The NWB file that we want is now stored locally. We need its filename to read it directly so that we don't have to use the AllenSDK any more.

In [ ]:
lfp_nwb_filename = os.path.join(local_cache_dir, f"session_{session_id}", f"probe_{probe_id}_lfp.nwb")
lfp_nwb_filename

Check the size of the file, should be about 2 GB.

In [ ]:
print(f"File size {os.path.getsize(lfp_nwb_filename) / 1024**3:.2f} GB")

## Read the NWB file

NWB files are stored in HDF5 format, so we can read it using h5py.

In [ ]:
import h5py

f = h5py.File(lfp_nwb_filename, "r")
f

Top-level keys in the file:

In [ ]:
f.keys()

There is lots of useful information here which can be examined by walking the tree structure. The LFP data is here:

In [ ]:
lfp_data = f[f"acquisition/probe_{probe_id}_lfp/probe_{probe_id}_lfp_data/data"]
lfp_data.shape, lfp_data.dtype, lfp_data.chunks

The first dimension is time and the second is channel. The corresponding timestamps are

In [ ]:
lfp_time = f[f"acquisition/probe_{probe_id}_lfp/probe_{probe_id}_lfp_data/timestamps"]
lfp_time.shape, lfp_time.dtype, lfp_time.chunks

There is also metadata for each array:

In [ ]:
print([item for item in lfp_data.attrs.items()], [item for item in lfp_time.attrs.items()])

although we are aren't using the metadata in this example workflow yet.

The chunk sizes are important here, as a chunk is the smallest unit of data that we can read and process. First note that the chunk sizes for the LFP data and timestamps are different, which is unfortunate. The chunk size of the data is quite small (41859*4 bytes = 164 kB) and the number of chunks of LFP data is

In [ ]:
print("Number of chunks", [s/c for s, c in zip(lfp_data.shape, lfp_data.chunks)])

So there are 256*93 = 23808 chunks in total. Possibly the chunk size in the time dimension has been chosen so that there are 256 chunks in this dimension?

# Create kerchunk reference JSON file

Now we use kerchunk to create a JSON file that separately refers to each chunk of LFP data in the NWB file and when loaded using zarr this behaves like a lazy-loaded 2D array without having to make a new copy of the data. We are constrained to use the same chunk size for the LFP data. We encode the 1D timestamp array within the JSON file; an alternative would be to reference the timestamps in the original NWB file but then there is the problem of their chunks being a different size to the LFP data. There are handy functions in kerchunk to process the whole of the NWB file, but we want to change the keys (i.e. xarray variable names) under which they are stored so we need to do some extra processing ourselves. The code for this is in the `nwb_kerchunk.py` Python file.

In [ ]:
from nwb_kerchunk import nwb_kerchunk

kerchunk_json_filename = "lfp.json"
nwb_kerchunk(probe_id, lfp_nwb_filename, kerchunk_json_filename)

# Load LFP data as an xarray using zarr

In [ ]:
import fsspec
import xarray as xr

In [ ]:
fs = fsspec.filesystem("reference", fo=kerchunk_json_filename)
xr_ds = xr.open_dataset(
    fs.get_mapper(),
    engine="zarr",
    backend_kwargs={"consolidated": False, "mask_and_scale": False}
)
xr_ds

To show this data is lazy-accessed, calculate the min and max of the first three channels:

In [ ]:
xr_ds.lfp[:, :3].min(axis=0).compute(), xr_ds.lfp[:, :3].max(axis=0).compute()

## Visualize using Datashader and Bokeh

Here is an example of how to interactively visualize the LFP data using just Datashader and Bokeh. Think of the code here as a simple HoloViews that has a callback when you try to zoom the Bokeh image that calls Datashader to re-datashade the new zoomed bounding box.

First let's look at a shorter timeseries, all channels (otherwise we have to load all of the data to obtain the min and max for each channel)

In [ ]:
xr_ds = xr_ds.isel(time=slice(0, 100_000))

Then calculate a new `DataArray` containing offset and scaled data, so that each channel is displayed offset with respect to the others.

In [ ]:
import numpy as np

channel_min = xr_ds["lfp"].min(axis=0)
channel_max = xr_ds["lfp"].max(axis=0)
xr_ds["lfp_subplot"] = (xr_ds["lfp"] - channel_min) / (channel_max - channel_min) + np.arange(xr_ds.dims["channel"])

Now create the Bokeh document with a Python callback to process RangesUpdate events.

In [ ]:
from bokeh.events import RangesUpdate
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
import datashader as ds

output_notebook()

def bkapp(doc):
    width = 1200
    height = 800
    canvas = ds.Canvas(plot_width=width, plot_height=height)
    agg = canvas.line(source=xr_ds, x="time", y="lfp_subplot", agg=ds.any())
    
    initialised = False
    x_range = agg.x_range
    y_range = agg.y_range
    tools = "pan, box_zoom, reset"
    p = figure(width=width, height=height, x_range=x_range, y_range=y_range, tools=tools)
    
    cds = ColumnDataSource(data=dict(
        image=[agg.data], x=[x_range[0]], y=[y_range[0]],
        dw=[x_range[1]-x_range[0]], dh=[y_range[1]-y_range[0]],
    ))
    
    im = p.image(source=cds, image="image", palette=["white", "blue"], x="x", y="y", dw="dw", dh="dh")
    
    def callback(event):
        global width, height, initialised
        if not initialised:
            # First call fixes datashader's canvas size to match the inner size of the figure
            width = p.inner_width
            height = p.inner_height
            initialised = True
    
        # Recalculate and render new image.
        canvas = ds.Canvas(plot_width=width, plot_height=height, x_range=(event.x0, event.x1), y_range=(event.y0, event.y1))
        agg = canvas.line(source=xr_ds, x="time", y="lfp_subplot", agg=ds.any())
        cds.data = dict(image=[agg.data], x=[event.x0], y=[event.y0], dw=[event.x1-event.x0], dh=[event.y1-event.y0])
    
    p.on_event(RangesUpdate, callback)
    
    doc.add_root(p)

Now run the Bokeh app and zoom and pan in the usual manner.

In [ ]:
show(bkapp)

## Multiple probes per dataset

In principle it is possible to combine the LFP data from multiple probes into a single kerchunked JSON file and hence a single zarr-loaded `xarray.Dataset`. The LFP would have a shape of `(nprobe, ntime, nchannel)`. The `time` for each probe is different, so the time coordinates would have to be 2D of shape `(nprobe, ntime)`. A bigger problem is that the LFP data of each channel has different chunk sizes. If they were the same then this approach would work, but if not then it is not possible. Dask chunking of multidimensional arrays allows the chunksize to vary as a function of its own dimension, but not as a function of other dimensions. Hence this idea is only possible if we copy the NWB data with it rechunked to a common chunk size. In future it would be preferable if the decision on how to chunk such data when it is stored takes this into account and uses a common chunk size for each probe in a session.